In [2]:

from collections import Counter
import numpy as np
from monitor import Monitor
import joblib
from data_model import DataModel
import tensorflow as tf
import os

model_src_list = []
model_args_list = []
father_model_name = './test_model/'
sum = 0
for i in range(10):
    model_list = [model_name.split('_') for model_name in os.listdir(father_model_name + str(i))]
    model_list = sorted(model_list, key=lambda s: s[1], reverse=True)
    model_name = model_list[0]
    model_args_list.append({
        'hidden': model_name[4],
        'full_hidden_num':model_name[7],
        'attention': model_name[10]
    })
    
    sum += float(model_name[1])
    model_name = "_".join(model_name)
    model_src_list.append(father_model_name + str(i) + '/' + model_name + '/')

print(sum / 10)

output_list = []
predict_intent_list = []
attention_rate_list = []

word2idx = joblib.load('./preprocessing_data/word2idx.pkl')
label2idx = joblib.load('./preprocessing_data/label2idx.pkl')
idx2vec = joblib.load('./preprocessing_data/idx2vec.pkl')
idx2label = joblib.load('./preprocessing_data/idx2label.pkl')

raw_data = [[], [], *DataModel(1).test_data]

for k_idx in range(10):
    model_src = model_src_list[k_idx]
    model_args = model_args_list[k_idx]
    sub_model = Monitor(data=raw_data, word2idx_dict=word2idx, label2idx_dict=label2idx,
                        idx2vec_dict=idx2vec, idx2label_dict=idx2label,
                        sentence_fixed_len=50, learning_rate=0.001, word_vec_size=400,
                        hidden_num=int(model_args['hidden']),full_hidden_num=int(model_args['full_hidden_num']), attention_num=int(model_args['attention']),
                        label_num=31, k_model_src=model_src)
    sub_model.load_model(model_src=model_src)
    output_list.append(sub_model.get_attention_information('', train_data_mode=False))

intent = output_list[0][1]
sentence = output_list[0][0]
sentence_length = output_list[0][2]
predict_intent = []
attention_rate = []

pre_int = []                        ############
pre_int_time = []                   ############
pre_att = []                        ############

for sentence_idx in range(len(sentence)):
    intent_time = []
    tmp_attention_rate = np.zeros(50)

    tmp_intent_time = -1            ############
    pre_int.append({})              ############
    pre_int_time.append({})         ############
    pre_att.append([])              ############

    for model_idx in range(len(output_list)):
        intent_time.append(output_list[model_idx][4][sentence_idx])
        tmp_attention_rate += output_list[model_idx][3][sentence_idx]

        if intent_time[-1] not in pre_int[-1]:                                      ############
            pre_int[-1][intent_time[-1]] = tmp_intent_time = tmp_intent_time + 1    ############
            pre_int_time[-1][intent_time[-1]] = 0                                   ############
            pre_att[-1].append(np.zeros(50))

        pre_int_time[-1][intent_time[-1]] += 1                                                  ############
        pre_att[-1][pre_int[-1][intent_time[-1]]] += output_list[model_idx][3][sentence_idx]    ############

    predict_intent.append(Counter(intent_time).most_common(1)[0][0])
    attention_rate.append([str(v) for v in list(tmp_attention_rate / len(output_list))])        ############
    for i_intent in pre_int[-1].keys():                                                         ############
        pre_att[-1][pre_int[-1][i_intent]] /= pre_int_time[-1][i_intent]                        ############






0.9477780000000001
INFO:tensorflow:Restoring parameters from ./test_model/0/acc_0.93947_lstm_hid_100_full_hid_0_att_hid_0_dropout_0.3_0.2_0.3/-46
INFO:tensorflow:Restoring parameters from ./test_model/1/acc_0.94474_lstm_hid_300_full_hid_0_att_hid_0_dropout_0.2_0.3_0.3/-30
INFO:tensorflow:Restoring parameters from ./test_model/2/acc_0.94974_lstm_hid_100_full_hid_0_att_hid_0_dropout_0.3_0.3_0.3/-46
INFO:tensorflow:Restoring parameters from ./test_model/3/acc_0.96552_lstm_hid_300_full_hid_0_att_hid_0_dropout_0.3_0.2_0.3/-42
INFO:tensorflow:Restoring parameters from ./test_model/4/acc_0.944_lstm_hid_100_full_hid_0_att_hid_0_dropout_0.3_0.2_0.3/-40
INFO:tensorflow:Restoring parameters from ./test_model/5/acc_0.93298_lstm_hid_150_full_hid_0_att_hid_0_dropout_0.3_0.2_0.3/-27
INFO:tensorflow:Restoring parameters from ./test_model/6/acc_0.95699_lstm_hid_100_full_hid_0_att_hid_0_dropout_0.2_0.3_0.3/-59
INFO:tensorflow:Restoring parameters from ./test_model/7/acc_0.97027_lstm_hid_150_full_hid_0_a

In [3]:
correct = 0
mistake = 0
with open('att_test_all_case.csv', 'w') as fp:
    for i in range(len(sentence)):
        if intent[i] == idx2label[predict_intent[i]]:
            correct += 1
        else:
            mistake += 1
        
        for i_intent in pre_int[i].keys():    
            fp.writelines((intent[i] + ',' + ','.join(sentence[i][:sentence_length[i]]) + ',\n'))
            fp.writelines((idx2label[i_intent]+ '(' + str(pre_int_time[i][i_intent]) + ')' + ',' +  
                           ','.join([str(j) 
                            for j in list(pre_att[i][pre_int[i][i_intent]][:sentence_length[i]])]) + 
                           ",\n"))   

print(correct)
print(correct / (mistake + correct))

with open('att_test_badcase.csv', 'w') as fp:
    for i in range(len(sentence)):
        if intent[i] == idx2label[predict_intent[i]]:
            correct += 1
        else:
            mistake += 1
            
            for i_intent in pre_int[i].keys():                 
                fp.writelines((intent[i] + ',' + ','.join(sentence[i][:sentence_length[i]]) + ',\n'))
                fp.writelines((idx2label[i_intent]+ '(' + str(pre_int_time[i][i_intent]) + ')' + ',' +
                               ','.join([str(j)
                                for j in list(pre_att[i][pre_int[i][i_intent]][:sentence_length[i]])]) + 
                                ",\n"))


3558
0.7857773851590106


In [1]:

from collections import Counter
import numpy as np
from monitor import Monitor
import joblib
from data_model import DataModel
import tensorflow as tf
import os

model_src_list = []
model_args_list = []
father_model_name = './test_model/'
model_list = []
sum = 0
for i in range(10):
    model_list += [[str(i)]+model_name.split('_') for model_name in os.listdir(father_model_name + str(i))]
for i in model_list:
    i.reverse()
model_list.sort()
i = 0
sum = 0
while (i + 10 <= len(model_list)):
    if (model_list[i][0:-3] == model_list[i + 9][0:-3]):
        tmp = 0
        for j in range(i, i + 10):
            tmp += float(model_list[j][-3])
        tmp /= 10
        if tmp > sum:
            sum = tmp;
            model_src_list = [model_list[j] for j in range(i,i + 10)]
        i += 10
    else:
        i += 1

print(sum)
print(model_src_list)

for i in range(len(model_src_list)):
    model_src_list[i].reverse()
    #print(model_src_list[i])
    model_args_list += [{'hidden': model_src_list[i][5],
                         'full_hidden_num' : model_src_list[i][8],
                         'attention' : model_src_list[i][11]}]
    model_src_list[i] = father_model_name + str(model_src_list[i][0]) + '/' + "_".join(model_src_list[i][1:])





output_list = []
predict_intent_list = []
attention_rate_list = []

word2idx = joblib.load('./preprocessing_data/word2idx.pkl')
label2idx = joblib.load('./preprocessing_data/label2idx.pkl')
idx2vec = joblib.load('./preprocessing_data/idx2vec.pkl')
idx2label = joblib.load('./preprocessing_data/idx2label.pkl')

# def load():
#     with open('./test_hyh.txt', encoding='utf8') as fp:
#         tmp = [v.strip() for v in fp.readlines()]
#     d = {}
#     for i, j in enumerate(tmp):
#         d[j] = i
#     return d

# d = load()
raw_data = [[], [], *DataModel(1).test_data]
#raw_data = [[], [], *DataModel(1).train_data]
for k_idx in range(10):
    model_src = model_src_list[k_idx]
    model_args = model_args_list[k_idx]
    sub_model = Monitor(data=raw_data, word2idx_dict=word2idx, label2idx_dict=label2idx,
                        idx2vec_dict=idx2vec, idx2label_dict=idx2label,
                        sentence_fixed_len=50, learning_rate=0.001, word_vec_size=400,
                        hidden_num=int(model_args['hidden']),full_hidden_num=int(model_args['full_hidden_num']), attention_num=int(model_args['attention']),
                        label_num=31, k_model_src=model_src)
    sub_model.load_model(model_src=model_src)
    output_list.append(sub_model.get_attention_information('', train_data_mode=False))

intent = output_list[0][1]
sentence = output_list[0][0]
sentence_length = output_list[0][2]
predict_intent = []
attention_rate = []

pre_int = []                        ############
pre_int_time = []                   ############
pre_att = []                        ############

for sentence_idx in range(len(sentence)):
    intent_time = []
    tmp_attention_rate = np.zeros(50)

    tmp_intent_time = -1            ############
    pre_int.append({})              ############
    pre_int_time.append({})         ############
    pre_att.append([])              ############

    for model_idx in range(len(output_list)):
        intent_time.append(output_list[model_idx][4][sentence_idx])
        tmp_attention_rate += output_list[model_idx][3][sentence_idx]

        if intent_time[-1] not in pre_int[-1]:                                      ############
            pre_int[-1][intent_time[-1]] = tmp_intent_time = tmp_intent_time + 1    ############
            pre_int_time[-1][intent_time[-1]] = 0                                   ############
            pre_att[-1].append(np.zeros(50))

        pre_int_time[-1][intent_time[-1]] += 1                                                  ############
        pre_att[-1][pre_int[-1][intent_time[-1]]] += output_list[model_idx][3][sentence_idx]    ############

    predict_intent.append(Counter(intent_time).most_common(1)[0][0])
    attention_rate.append([str(v) for v in list(tmp_attention_rate / len(output_list))])        ############
    for i_intent in pre_int[-1].keys():                                                         ############
        pre_att[-1][pre_int[-1][i_intent]] /= pre_int_time[-1][i_intent]                        ############

        

correct = 0
mistake = 0
ans = []

with open('att_test_all_case.csv', 'w') as fp:
    for i in range(len(sentence)):
        if intent[i] == idx2label[predict_intent[i]]:
            correct += 1
        else:
            mistake += 1
        
        mx = 0
        output = ""
        for i_intent in pre_int[i].keys():
            if pre_int_time[i][i_intent] > mx:
                mx = pre_int_time[i][i_intent]
                output = (idx2label[i_intent]+ '(' + str(pre_int_time[i][i_intent]) + ')' + ',' +
                           ','.join([str(j)
                            for j in list(pre_att[i][pre_int[i][i_intent]][:sentence_length[i]])]) + 
                            ",\n")
#         ans = [ d[''.join(sentence[i][:sentence_length[i]])], 
#                (intent[i] + ',' + ','.join(sentence[i][:sentence_length[i]]) + ',\n'), 
#                output ]
        fp.writelines((intent[i] + ',' + ','.join(sentence[i][:sentence_length[i]]) + ',\n'))
        fp.writelines(output)
#     ans.sort()
#     for i in ans:
#         fp.writelines(ans[1])
#         fp.writelines(ans[2])
        
#         for i_intent in pre_int[i].keys():    
#            fp.writelines((intent[i] + ',' + ','.join(sentence[i][:sentence_length[i]]) + ',\n'))
#            fp.writelines((idx2label[i_intent]+ '(' + str(pre_int_time[i][i_intent]) + ')' + ',' +  
#                           ','.join([str(j) 
#                            for j in list(pre_att[i][pre_int[i][i_intent]][:sentence_length[i]])]) + 
#                           ",\n"))   
    
print(correct)
print(correct / (mistake + correct))

with open('att_test_badcase.csv', 'w') as fp:
    for i in range(len(sentence)):
        if intent[i] == idx2label[predict_intent[i]]:
            correct += 1
        else:
            mistake += 1
            fp.writelines((intent[i] + ',' + ','.join(sentence[i][:sentence_length[i]]) + ',\n'))
            mx = 0
            output = ""
            for i_intent in pre_int[i].keys():
                if pre_int_time[i][i_intent] > mx:
                    mx = pre_int_time[i][i_intent]
                    output = (idx2label[i_intent]+ '(' + str(pre_int_time[i][i_intent]) + ')' + ',' +
                               ','.join([str(j)
                                for j in list(pre_att[i][pre_int[i][i_intent]][:sentence_length[i]])]) + 
                                ",\n")
            fp.writelines(output)
            


Using TensorFlow backend.


0.9405660000000001
[['0.3', '0.3', '0.3', 'dropout', '0', 'hid', 'att', '0', 'hid', 'full', '300', 'hid', 'lstm', '0.92761', 'acc', '5'], ['0.3', '0.3', '0.3', 'dropout', '0', 'hid', 'att', '0', 'hid', 'full', '300', 'hid', 'lstm', '0.92857', 'acc', '9'], ['0.3', '0.3', '0.3', 'dropout', '0', 'hid', 'att', '0', 'hid', 'full', '300', 'hid', 'lstm', '0.93421', 'acc', '0'], ['0.3', '0.3', '0.3', 'dropout', '0', 'hid', 'att', '0', 'hid', 'full', '300', 'hid', 'lstm', '0.93651', 'acc', '2'], ['0.3', '0.3', '0.3', 'dropout', '0', 'hid', 'att', '0', 'hid', 'full', '300', 'hid', 'lstm', '0.93733', 'acc', '8'], ['0.3', '0.3', '0.3', 'dropout', '0', 'hid', 'att', '0', 'hid', 'full', '300', 'hid', 'lstm', '0.93867', 'acc', '4'], ['0.3', '0.3', '0.3', 'dropout', '0', 'hid', 'att', '0', 'hid', 'full', '300', 'hid', 'lstm', '0.93947', 'acc', '1'], ['0.3', '0.3', '0.3', 'dropout', '0', 'hid', 'att', '0', 'hid', 'full', '300', 'hid', 'lstm', '0.94892', 'acc', '6'], ['0.3', '0.3', '0.3', 'dropout', '0'

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.602 seconds.
Prefix dict has been built succesfully.


INFO:tensorflow:Restoring parameters from ./test_model/5/acc_0.92761_lstm_hid_300_full_hid_0_att_hid_0_dropout_0.3_0.3_0.3/-8
INFO:tensorflow:Restoring parameters from ./test_model/9/acc_0.92857_lstm_hid_300_full_hid_0_att_hid_0_dropout_0.3_0.3_0.3/-60
INFO:tensorflow:Restoring parameters from ./test_model/0/acc_0.93421_lstm_hid_300_full_hid_0_att_hid_0_dropout_0.3_0.3_0.3/-25
INFO:tensorflow:Restoring parameters from ./test_model/2/acc_0.93651_lstm_hid_300_full_hid_0_att_hid_0_dropout_0.3_0.3_0.3/-40
INFO:tensorflow:Restoring parameters from ./test_model/8/acc_0.93733_lstm_hid_300_full_hid_0_att_hid_0_dropout_0.3_0.3_0.3/-22
INFO:tensorflow:Restoring parameters from ./test_model/4/acc_0.93867_lstm_hid_300_full_hid_0_att_hid_0_dropout_0.3_0.3_0.3/-23
INFO:tensorflow:Restoring parameters from ./test_model/1/acc_0.93947_lstm_hid_300_full_hid_0_att_hid_0_dropout_0.3_0.3_0.3/-45
INFO:tensorflow:Restoring parameters from ./test_model/6/acc_0.94892_lstm_hid_300_full_hid_0_att_hid_0_dropout_0